<a href="https://colab.research.google.com/github/srinivasrdhkrshnn/CS6910-Assignment-4/blob/main/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb

     |████████████████████████████████| 1.8MB 27.0MB/s 
     |████████████████████████████████| 133kB 52.0MB/s 
     |████████████████████████████████| 102kB 14.8MB/s 
     |████████████████████████████████| 174kB 50.5MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=4702697a5c4807b4afed44ccd2fbc92a516bc9e1c0d65aff251cef54d1cde3da
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=4333cb94fc68b966368f4fc7c6dc3e9be2c3fa4bf3c19c5a2c5040cda7ec667b
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from keras.datasets import mnist
from sklearn.manifold import TSNE
import wandb

# load dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
class_type = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat','Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] 

proj_name='CS6910 Assignment 4'

In [6]:
# Initializations

# Data
X_train = np.array(x_train.reshape(x_train.shape[0], 784,1))         # reshape 2-D data to 1-D
X_test  = np.array(x_test.reshape(x_test.shape[0], 784,1))           # reshape 2-D data to 1-D
X_train = (X_train > 126) * 1                                        # convert the real valued data into binary data, using a threshold of 127
X_test  = (X_test > 126) * 1                                         # convert the real valued data into binary data, using a threshold of 127


Y_train = np.zeros([len(y_train),10,1])
Y_test = np.zeros([len(y_test),10,1])

for i in range(len(y_train)):                                        # convert y from just a class number to one hot vector (10x1)
  y = np.zeros([10, 1])
  y[y_train[i]] = 1.0
  Y_train[i] = y

for i in range(len(y_test)):                                         # convert y from just a class number to one hot vector (10x1)
  y = np.zeros([10, 1])
  y[y_test[i]] = 1.0
  Y_test[i] = y                                                      # test set output



In [31]:
n_visible = X_train.shape[1]                                         # number of visible neurons
n_examples = X_train.shape[0]                                        # number of training data
n_hidden = 80                                                        # number of hidden neurons
n_class = 10

epochs = 5                                                           # number of epochs for training
k = 40                                                               # number of steps , Markov chain is run
learning_rate = 0.1                                                  # learning rate of the update
batchsize = 100                   

***Parameter Initialization***

In [30]:
def rbm_param_init() :                                                                        # Function to randomly initialize the parameters 
	
  rbm_parameters = {}
  rbm_parameters["W"] = np.random.randn(n_hidden, n_visible)*np.sqrt(6./(n_visible + n_hidden))   # Xavier Initialization of weights
  rbm_parameters["h_bias"] = np.zeros((n_hidden,1),dtype=np.float64)                                                     
  rbm_parameters["v_bias"] = np.zeros((n_visible,1),dtype=np.float64)

  return rbm_parameters

def classifier_param_init() :

  classifier = {}
  classifier["W"] = np.random.randn(n_class, n_hidden)*np.sqrt(6./(n_class + n_hidden))       # Xavier Initialization of weights
  classifier["b"] = np.zeros((n_class,1),dtype=np.float64)

  return classifier

def sigmoid(x) :                                                                              # activation function 
	
  return 1.0/(1.0+np.exp(-x))

In [36]:
def rbm_train(X_train,parameters,k,learning_rate,epochs) :                                     # Function to train the RBM
	
  W = parameters["W"]
  h_bias = parameters["h_bias"]
  v_bias = parameters["v_bias"]
  
  #batches = n_examples // batchsize

  for epoch in range(epochs) :
     for i in range(n_examples) :
        v_sample = X_train[i]
        v_init   = X_train[i]

        for t in range(k) :                                                               # Markov Chain - Loop
          h_given_v = sigmoid(np.dot(W,v_sample)+h_bias)                                  # Evaluate p(h|v)
          h_sample = np.random.binomial(1,h_given_v)                                      # Convert to 0's and 1's assuming binomial distribution p(h|v) 
          v_given_h = sigmoid(np.dot(np.transpose(W),h_sample)+v_bias)                    # Evaluate p(v|h)
          v_sample = np.random.binomial(1,v_given_h)                                      # Convert to 0's and 1's assuming binomial distribution p(v|h)

        # Update Rule
        W = W + learning_rate*(np.dot(sigmoid(np.dot(W,v_init)+h_bias),np.transpose(v_init)) - np.dot(sigmoid(np.dot(W,v_sample)+h_bias),np.transpose(v_sample)))
        v_bias = v_bias + learning_rate*(v_init-v_sample)
        h_bias = h_bias + learning_rate*(sigmoid(np.dot(W,v_init)+h_bias) - sigmoid(np.dot(W,v_sample)+h_bias))

  parameters["W"] = W
  parameters["h_bias"] = h_bias
  parameters["v_bias"] = v_bias
  print("Training Complete")

  return parameters


def get_hidden(x,parameters) :                                                          # function to get the hidden representation of the test data
   
    W = parameters["W"]
    h_bias = parameters["h_bias"]
    hidden_prob = sigmoid(np.dot(W,x)+h_bias)
    hidden_rep = np.random.binomial(1,hidden_prob)
    
    return hidden_rep

def classifier(X,Y,rbm_parameters)  :                                                   # Function to train Classifier
    
    classifier_param = classifier_param_init()
    W = classifier_param["W"]
    b = classifier_param["b"]

    for i in range(n_examples) :
         # feed forward
         hidden_rep = get_hidden(X[i],rbm_parameters)
         pre_output = np.dot(W,hidden_rep)+b
         y_hat = softmax(pre_output)
         # backpropogate
         dW = np.dot(-(Y[i]-y_hat),np.transpose(X[i]))
         db = -(Y[i]-y_hat)
         # Update Classifier weights
         W = W - learning_rate*dW
         b = b - learning_rate*dB

    classifier_param["W"] = W
    classifier_param["b"] = b   

return  classifier_param   

    
    



In [ ]:
for k in [1] :
	rbm_parameters = rbm_param_init()
	rbm_parameters = rbm_train(X_train,rbm_parameters,k,learning_rate,epochs)
  classifier_param = classifier(X_train,Y_train,rbm_parameters)
  
	

In [38]:
a = np.array([[[1],[2],[3]],[[2],[3],[4]]])
b = a[1]
print(b)

for k in [2]:
  z = k+1
  print(z)

[[2]
 [3]
 [4]]
3
